In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math
import matplotlib.animation as ani
import cv2
from datetime import datetime

from __future__ import print_function
import argparse

def graphFunc(time, frame, height_profile):
    max_height = max(height_profile)
    len_height = len(height_profile)
    
    if time > len_height:
        return True
    plt.clf()
    # for '/climbs/0/height_profile' we are sampling at 10Hz, i.e. every 0.1s
    ind = min(math.floor(time/0.1), len_height) # stay in bounds
    plt.plot(height_profile[:math.floor(time/0.1)], linewidth=7.0, alpha=0.5)
    plt.ylim(0,max_height)
    plt.xlim(0,len_height)
    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.axes.xaxis.set_visible(False)
    
    plt.savefig('plot.png', transparent=True)

    # imread_unchange reads plot.png with 4 channels, i.e. including alpha channel which
    # describes the transparancy
    overlay = cv2.imread('plot.png', cv2.IMREAD_UNCHANGED)
    new_frame = frame

    y_offset = x_offset = 5
    y1, y2 = y_offset, y_offset + overlay.shape[0]
    x1, x2 = x_offset, x_offset + overlay.shape[1]
    alpha_o = overlay[:, :, 3] / 255.0
    alpha_n = 1.0 - alpha_o

    for c in range(0, 3):
        new_frame[y1:y2, x1:x2, c] = (alpha_o * overlay[:, :, c] +
                                  alpha_n * new_frame[y1:y2, x1:x2, c])
    
    return new_frame

# playback speed = out_fps/30 * speed_up   
# speed_up is positive int, e.g. 2 means skip every 2nd frame, 3 skip every 3rd frame
def overlayGraph(videofile, h5_file, outfile, graphFunc, out_fps, speed_up):
    f = h5py.File(h5_file_name, 'r')
    height_profile = f['climbs/0/height_profile']
    
    cap = cv2.VideoCapture(videofile)
    
    width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    writer= cv2.VideoWriter(outfile, cv2.VideoWriter_fourcc(*'MP4V'), out_fps, (width,height))    

    """ Set figure size, so it's not bigger than video """
    figure(num=None, figsize=((width - 20)/80, (height - 20)/80), dpi=80)
    i = 0 # count which frame we are on
    while True:
        if not (i % speed_up == 0):
            cap.read()
            i+=1
            continue
        ret,frame = cap.read()

        if not ret:
            break
        
        time = i / 30;
        new_frame = graphFunc(time, frame, height_profile)

        writer.write(new_frame)
        cv2.imshow('frame', new_frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break
            
        i+=1

    cap.release()
    writer.release()
    cv2.destroyAllWindows()

h5_file_name = '../data/overhang/bae8f52c-407e-5f89-a8e3-61fcca51ee0a.h5'

overlayGraph("../data/overhang/Overhang (vertical + horizontal).mp4", h5_file_name, 'graphvid.mp4', graphFunc, 45, 2)